In [3]:
from PIL import Image
import numpy as np
from collections import Counter
import colorsys

# Extended color-to-note mapping with semitones
COLOR_TO_NOTE = {
    'C': (255, 0, 0),          # Red
    'C#': (255, 102, 102),     # Light Red
    'D': (255, 165, 0),        # Orange
    'D#': (255, 140, 0),       # Dark Orange
    'E': (255, 255, 0),        # Yellow
    'F': (0, 255, 0),          # Green
    'F#': (102, 255, 102),     # Light Green
    'G': (0, 0, 255),          # Blue
    'G#': (102, 178, 255),     # Light Blue
    'A': (75, 0, 130),         # Indigo
    'A#': (238, 130, 238),     # Violet
    'B': (186, 85, 211)        # Light Purple
}

def calculate_average_brightness(image):
    """
    Calculate the average brightness of an image.
    Converts the image to grayscale and computes the mean pixel value.
    """
    grayscale_image = image.convert('L')  # Convert to grayscale
    pixels = np.array(grayscale_image)
    average_brightness = np.mean(pixels)
    return average_brightness


def resize_image(image, max_size=500):
    """Reduz o tamanho da imagem para melhorar a performance."""
    image.thumbnail((max_size, max_size), Image.BILINEAR)
    return image


def calculate_saturation(image):
    """
    Calculate the average saturation of an image.
    Converts the image to RGB, then to HSV to extract saturation values.
    """
    rgb_image = image.convert('RGB')  # Ensure image is in RGB
    pixels = np.array(rgb_image)
    # Reshape to a list of pixels
    reshaped_pixels = pixels.reshape(-1, 3)
    # Convert RGB to HSV and extract saturation
    hsv_pixels = [colorsys.rgb_to_hsv(r/255.0, g/255.0, b/255.0) for r, g, b in reshaped_pixels]
    saturations = [s for h, s, v in hsv_pixels]
    average_saturation = np.mean(saturations)
    return average_saturation

def find_nearest_note_color(pixel, color_map):
    """
    Find the nearest note color for a given pixel based on Euclidean distance.
    """
    min_distance = float('inf')
    nearest_note = None
    for note, color in color_map.items():
        distance = np.linalg.norm(np.array(pixel) - np.array(color))  # Euclidean distance
        if distance < min_distance:
            min_distance = distance
            nearest_note = note
    return nearest_note

def find_rainbow_colors(image, color_map):
    """
    Find and count the frequency of each rainbow color (including semitones) in the image.
    """
    rgb_image = image.convert('RGB')  # Ensure image is in RGB
    pixels = np.array(rgb_image)
    # Reshape to a list of pixels
    reshaped_pixels = pixels.reshape(-1, 3)
    pixel_counter = Counter(map(tuple, reshaped_pixels))

    # Initialize frequency dictionary
    color_frequency = {note: 0 for note in color_map.keys()}

    for pixel, count in pixel_counter.items():
        note = find_nearest_note_color(pixel, color_map)
        if note:
            color_frequency[note] += count

    # Order colors by frequency
    ordered_colors = sorted(color_frequency.items(), key=lambda x: x[1], reverse=True)
    return ordered_colors

def assign_music_features(brightness, ordered_colors, saturation):
    """
    Assign pitch, tone sequence, and tempo based on image features.
    """
    # Assign pitch based on brightness (A4 = 440Hz is middle range)
    base_pitch = 440  # Middle A pitch
    pitch = int(base_pitch * (brightness / 255))  # Scale brightness to pitch range

    # Assign sequence of tones based on color order (already mapped to notes)
    tone_sequence = [note for note, freq in ordered_colors[:4] if freq > 0]

    # Assign tempo based on saturation (higher saturation, faster tempo)
    base_tempo = 240  # Base tempo in BPM
    tempo = int(base_tempo * saturation)  # Scale saturation to tempo
    if tempo > 160:
        tempo = 160

    return pitch, tone_sequence, tempo

def process_image(image_path, resize = True):
    """
    Process the image to extract features and assign musical attributes.
    """
    try:
        image = Image.open(image_path)
    except IOError:
        print(f"Error: Unable to open image at path '{image_path}'. Please check the file path.")
        return
    
    if resize == True:
        image = resize_image(image)

    # Extract features
    brightness = calculate_average_brightness(image)
    saturation = calculate_saturation(image)
    ordered_colors = find_rainbow_colors(image, COLOR_TO_NOTE)

    # Assign music features
    pitch, tone_sequence, tempo = assign_music_features(brightness, ordered_colors, saturation)

    # Print the results
    print(f"Average Brightness: {brightness:.2f}")
    print("Color Frequencies (Ordered):")
    for color, freq in ordered_colors:
        print(f"  {color}: {freq}")
    print(f"Average Saturation: {saturation:.2f}")
    print(f"Assigned Pitch (Hz): {pitch}")
    print(f"Tone Sequence: {' - '.join(tone_sequence)}")
    print(f"Assigned Tempo (BPM): {tempo}")

    return {
        "brightness": brightness,
        "ordered_colors": ordered_colors,
        "saturation": saturation,
        "pitch": pitch,
        "tone_sequence": tone_sequence,
        "tempo": tempo
    }

if __name__ == "__main__":
    # Example usage
    image_path = 'james_webb_images/heic0004_large.jpg'  # Replace with your image path
    features = process_image(image_path)

Average Brightness: 5.98
Color Frequencies (Ordered):
  A: 246569
  B: 1051
  G#: 913
  F#: 665
  A#: 517
  F: 172
  C#: 112
  D#: 1
  C: 0
  D: 0
  E: 0
  G: 0
Average Saturation: 0.92
Assigned Pitch (Hz): 10
Tone Sequence: A - B - G# - F#
Assigned Tempo (BPM): 160


In [4]:
music_features = [features[feature] for feature in ['tone_sequence', 'tempo']]

In [5]:
import requests
import json
import os

api_key = 'sk-FDti6mHcD9tIT6bNgmrHvPd9bUcUqCJRfJLhynB9OkT3BlbkFJSvdRzIA5BrDL_GKIhBkFTKl-fZKmmSYoUMizf8ZEMA'
input_text = 'In the cosmic expanse, a nebula blooms, Swirls of gas and dust like a dream in hues. Golden cliffs rise, sculpted by stellar winds, Reaching into the void where eternity begins.'

tone_sequence = music_features[0]
tempo = music_features[1]

class SongMaker:
    def __init__(self, api_key, input_text):
        self.api_key = api_key
        self.input_text = input_text
        self.music_features = music_features
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }


    def make_song(self):
        """Makes an ABC format song based on a input text."""
        payload = {
            "model": "gpt-4o-mini",
            "messages": [
                {
                    "role":"system",
                    "content": "You are DeveloperBot, powered by GPT-4, a large language model trained by OpenAI. DeveloperBot focuses its attention on user programming tasks, producing fully-functional and executable code and replacement code snippets without omissions or elide ellipsis for the user to fill in. Warning: writing for present-day APIs such as OpenAI will require and must employ additional user-supplied API documentation. You are especially useful in writing good, creative, usable, stable, correct ABC music."
                },
                {
                    "role": "user",
                    "content": "RolePlay as a musical bot. Generate a a music in ABC format based on the input text: {self.input_text}"
                },
                {
                    "role": "user",
                    "content": """ Follows an example of a well-formated abc file:
                    
                    
                    X: 1
T: Star Wars Main Theme
C: John Williams
Q: "Jedi-Like"
O: from Jenny O'Connor tutorial
R: march
Z: 2016 John Chambers <jc:trillian.mit.edu>
S: www.thehotviolinist.com
M: 4/4
L: 1/8
K: G
(3uDDD |\
vG4 d4 | (3vcBA g4 d2 |\
(3ucBA g4 d2 | (3vcBc uA4 :|\
vD>D |\
vE3E cBAG | (3GAB AE F2 vD>D |
E4E cBAG | d2 A4 vD>D |\
vE3E cBAG | (3GAB AE F2 d>d |\
(3:2:2g2=f (3:2:2_e2d (3:2:2c2_B (3:2:2A2G |\
d6 uDDD | vd8 |]"""
                },
                {
                    "role": "user",
                    "content": f"Style: Symphonic, in the style of Hans Zimmer. Aim for a longer sequence, of about a minute."
                },
                {
                    "role": "user",
                    "content": "Tones should include: {tone_sequence}. Tempo: {tempo} BPM. Pitch: Low-pitch. Aim for a very inspirational song, that motivates exploration."
                },
                {
                    "role": "user",
                    "content": "Make a melodic synphony."
                },
                {
                    "role": "user",
                    "content": "Remove any introductions or explanations. Send only the requested text."
                },
            ],
            "max_tokens": 600
        }

        response = requests.post("https://api.openai.com/v1/chat/completions",
                                 headers=self.headers, json=payload)
        return response.json()


generate_song = SongMaker(api_key, input_text)

# Make a song
song = generate_song.make_song()
print(song)

{'id': 'chatcmpl-AFSJNzpGuv0k7qJbOrEihEzMQwACa', 'object': 'chat.completion', 'created': 1728247113, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```\nX: 1\nT: Symphony of Exploration\nC: Inspired by Hans Zimmer\nQ: 120\nO: User-generated\nR: orchestral\nZ: 2023 DeveloperBot\nM: 4/4\nL: 1/8\nK: C\n"Low-Pitch" A4 G4 E4 | F4 E4 C4 D4 | C4 (3EGB A4 | (3GAB c4 d4 | (3Bcd e3g d4 | \n(3BcB A2G2 | (3EGB c4 A4 | G4 D4 C2D2 :| \nV: 2 \n"Strings" D4 B2 G2 | F3E D4 C4 | (3Bcd e3g c4 | (3dBA B2 A2 | F4 E4 C2D2 | \n(3EGB A4 F4 | B4 D4 C2G2 | (3EGB A4 C4 | \nV: 3 \n"Brass" G4 D4 C4 | F4 E4 G4 A4 | (3dB^c g4 a4 | (3gfe e2D2 | C4 B4 A4 G4 | \n(3GFE D4 C4 | E4 G4 F2D2 | B4 D4 C2C2 | \nV: 1,2,3 \n"Full Orchestra" C8 | C4 D4 E4 F4 | (3EGB A4 c4 | (3Bcd e3g d4 | \nG4 A4 C2C2 | F4 E4 D4 C4 | G4 E4 C8 | C2C2 z8 |]\n```', 'refusal': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 463, 'completion_tokens': 382, 'total_t

In [6]:
song

{'id': 'chatcmpl-AFSJNzpGuv0k7qJbOrEihEzMQwACa',
 'object': 'chat.completion',
 'created': 1728247113,
 'model': 'gpt-4o-mini-2024-07-18',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '```\nX: 1\nT: Symphony of Exploration\nC: Inspired by Hans Zimmer\nQ: 120\nO: User-generated\nR: orchestral\nZ: 2023 DeveloperBot\nM: 4/4\nL: 1/8\nK: C\n"Low-Pitch" A4 G4 E4 | F4 E4 C4 D4 | C4 (3EGB A4 | (3GAB c4 d4 | (3Bcd e3g d4 | \n(3BcB A2G2 | (3EGB c4 A4 | G4 D4 C2D2 :| \nV: 2 \n"Strings" D4 B2 G2 | F3E D4 C4 | (3Bcd e3g c4 | (3dBA B2 A2 | F4 E4 C2D2 | \n(3EGB A4 F4 | B4 D4 C2G2 | (3EGB A4 C4 | \nV: 3 \n"Brass" G4 D4 C4 | F4 E4 G4 A4 | (3dB^c g4 a4 | (3gfe e2D2 | C4 B4 A4 G4 | \n(3GFE D4 C4 | E4 G4 F2D2 | B4 D4 C2C2 | \nV: 1,2,3 \n"Full Orchestra" C8 | C4 D4 E4 F4 | (3EGB A4 c4 | (3Bcd e3g d4 | \nG4 A4 C2C2 | F4 E4 D4 C4 | G4 E4 C8 | C2C2 z8 |]\n```',
    'refusal': None},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 463,
  'completio

In [7]:
print(song['choices'][0]['message']['content'][3:][:-4])


X: 1
T: Symphony of Exploration
C: Inspired by Hans Zimmer
Q: 120
O: User-generated
R: orchestral
Z: 2023 DeveloperBot
M: 4/4
L: 1/8
K: C
"Low-Pitch" A4 G4 E4 | F4 E4 C4 D4 | C4 (3EGB A4 | (3GAB c4 d4 | (3Bcd e3g d4 | 
(3BcB A2G2 | (3EGB c4 A4 | G4 D4 C2D2 :| 
V: 2 
"Strings" D4 B2 G2 | F3E D4 C4 | (3Bcd e3g c4 | (3dBA B2 A2 | F4 E4 C2D2 | 
(3EGB A4 F4 | B4 D4 C2G2 | (3EGB A4 C4 | 
V: 3 
"Brass" G4 D4 C4 | F4 E4 G4 A4 | (3dB^c g4 a4 | (3gfe e2D2 | C4 B4 A4 G4 | 
(3GFE D4 C4 | E4 G4 F2D2 | B4 D4 C2C2 | 
V: 1,2,3 
"Full Orchestra" C8 | C4 D4 E4 F4 | (3EGB A4 c4 | (3Bcd e3g d4 | 
G4 A4 C2C2 | F4 E4 D4 C4 | G4 E4 C8 | C2C2 z8 |]


In [9]:
# Let's attempt to convert the revised ABC notation to MIDI using a different approach
from music21 import converter, midi
# Define the revised ABC notation
abc_code = song['choices'][0]['message']['content'][3:][:-4].replace('\n', '')

# Attempt to convert the ABC notation to MIDI using music21
def make_midi(abc_code):
    try:

        score = converter.parse(abc_code, format='abc')

    # Step 4: Write to a MIDI file
        midi_filename = "cosmic_bloom.mid"
        score.write('midi', fp=midi_filename)

        print(f"MIDI file saved as {midi_filename}")
        return True
    except Exception as e:
        print(str(e))
        return False

In [10]:
make_midi(abc_code)

invalid literal for int() with base 10: '1T: Symphony of ExplorationC: Inspired by Hans ZimmerQ: 120O: User-generatedR: orchestralZ: 2023 DeveloperBotM: 4/4L: 1/8K: C"Low-Pitch" A4 G4 E4 | F4 E4 C4 D4 | C4 (3EGB A4 | (3GAB c4 d4 | (3Bcd e3g


False

In [11]:
def make_midi_image(api_key, input_text, music_features):
    finished = False
    tone_sequence = music_features[0]
    tempo = music_features[1]

    while finished == False:
        song = SongMaker(api_key, input_text).make_song()
        song_abc = song['choices'][0]['message']['content'][3:][:-4].replace('\n', '''
                                                                             ''')
        finished = make_midi(song_abc)

In [12]:
make_midi_image(api_key, input_text, music_features)

the object (<music21.meter.TimeSignature 4/4>, id()=140676701590992 is already found in this Stream (<music21.stream.Part 0x7ff1d91cb7c0>, id()=140676706383808)
the object (<music21.meter.TimeSignature 4/4>, id()=140676699219200 is already found in this Stream (<music21.stream.Part 0x7ff1d8ce30d0>, id()=140676701237456)
MIDI file saved as cosmic_bloom.mid


In [15]:
import os
import subprocess
from pydub import AudioSegment
from pydub.effects import normalize

# Step 1: Convert MIDI to WAV using FluidSynth
def midi_to_wav(midi_file, soundfont, output_file):
    """
    Converts a MIDI file to WAV using FluidSynth.
    """
    # FluidSynth command to convert MIDI to WAV
    command = [
        "fluidsynth",
        "-ni", soundfont,  # Use the specified soundfont
        midi_file,
        "-F", output_file,  # Output WAV file
        "-r", "44100"  # Sample rate
    ]
    
    # Execute the command
    subprocess.run(command, check=True)

# Step 2: Apply reverb using sox
def apply_reverb(wav_file, output_file):
    """
    Applies reverb to a WAV file using the sox tool.
    """
    # Sox command to apply reverb
    command = [
        "sox", wav_file, output_file, "reverb"
    ]
    
    # Execute the command
    subprocess.run(command, check=True)

# Step 3: Apply normalization and other effects
def apply_effects(wav_file, output_file):
    """
    Normalizes the audio file after reverb.
    """
    # Load the WAV file
    audio = AudioSegment.from_wav(wav_file)
    
    # Normalize the audio (optional but ensures even levels)
    normalized_audio = normalize(audio)
    
    # Export the final output to WAV
    normalized_audio.export(output_file, format="wav")

# Step 4: Convert MIDI to WAV, apply reverb and normalization
def process_midi(midi_input, soundfont, output_wav):
    temp_wav = "temp_output.wav"  # Temporary file to hold the initial WAV output
    temp_with_reverb = "temp_with_reverb.wav"  # Temporary file for reverb

    # Convert MIDI to WAV
    midi_to_wav(midi_input, soundfont, temp_wav)

    # Apply reverb using sox
    apply_reverb(temp_wav, temp_with_reverb)

    # Apply normalization and export as final WAV
    apply_effects(temp_with_reverb, output_wav)

    # Clean up temporary files
    os.remove(temp_wav)
    os.remove(temp_with_reverb)
# Example usage

midi_file = "cosmic_bloom.mid"
output_wav_file = "output_song_with_effects.wav"
soundfont = "Orchestral Essentials.SF2"

# Process the MIDI file and add effect
process_midi(midi_file, soundfont, output_wav_file)

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'temp_output.wav'..
